In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from os import listdir
from os.path import join,exists
from tqdm import tnrange
from util import *

%matplotlib inline

with open('raw/tracks_uri_2_id.json') as f:
    tracks_uri_2_id=json.load(f)
with open('raw/tracks_id_2_uri.json') as f:
    tracks_id_2_uri=json.load(f)

def load_small_data():
    playlists,playlist_tracks,tracks=load_train_data('small')
    playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,playlist_tracks_tru=split_data(playlists,playlist_tracks,test_size=0.01)
    return playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,playlist_tracks_tru,tracks

def load_large_data():
    return pd.read_csv('raw/large/playlists_tra.csv'),\
pd.read_csv('raw/large/playlists_tst.csv'),\
pd.read_csv('raw/large/playlist_tracks_tra.csv'),\
pd.read_csv('raw/large/playlist_tracks_tst.csv'),\
pd.read_csv('raw/large/playlist_tracks_tru.csv'),\
pd.read_csv('raw/large/tracks.csv')

def load_data():
    return pd.read_csv('raw/large/playlists.csv'),\
pd.read_csv('raw/test/playlists.csv'),\
pd.read_csv('raw/large/playlist_tracks.csv'),\
pd.read_csv('raw/test/playlist_tracks.csv'),\
pd.read_csv('raw/large/tracks.csv')

playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,playlist_tracks_tru,tracks = load_small_data()
y_true=ytrue(playlists_tst,playlist_tracks_tru)

# playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,y_true = load_large_data()
# playlists=pd.concat([playlists_tra,playlists_tst])
# pids_tra=set(playlists_tra.pid)
# assert playlists.shape[0]==playlists_tra.shape[0]+playlists_tst.shape[0]


playlists_tst.info()

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 90112 to 427007
Data columns (total 11 columns):
collaborative    100 non-null bool
description      2 non-null object
duration_ms      100 non-null int64
modified_at      100 non-null int64
name             80 non-null object
num_albums       100 non-null int64
num_artists      100 non-null int64
num_edits        100 non-null int64
num_followers    100 non-null int64
num_tracks       100 non-null int64
pid              100 non-null int64
dtypes: bool(1), int64(8), object(2)
memory usage: 8.7+ KB


In [4]:
%%writefile title.py
import re
import numpy as np
import pandas as pd
from tqdm import tnrange

pattern = re.compile('[\W_]+')
def build_title(name):
    n=str(name).lower()
    words=n.split()
    for i,w in enumerate(words):
        words[i]=pattern.sub('', w)
    w=''.join(words)
    return w

def recommend(playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,tracks,proc_i=-1,beg=-1,end=-1):
    calc_sim_n=0
    np.random.seed=42

    pids=sorted(playlists_tst.pid)
    if proc_i>-1:
        pids=pids[beg:end]
    y_pred={}

    rc=500
    
    sim_cnts=[]
    for i in tnrange(len(pids)):
        pid_tst=pids[i]
        pred_tracks=np.zeros((rc,))
        playlists_sim = playlists_tra[playlists_tra.title==playlists_tst.iloc[i]['title']]

        sim_cnts.append(len(playlists_sim))
        j=0
        seed_tracks=set(playlist_tracks_tst[playlist_tracks_tst.pid == pid_tst].track_id)

        if len(playlists_sim)>0:
            tracks_tra=playlist_tracks_tra.join(playlists_sim,how='inner',rsuffix='_other').sort_values('pos')[['track_id']].drop_duplicates()
            
            for idx in range(tracks_tra.shape[0]):
                t = tracks_tra.iloc[idx]['track_id']
                if t not in seed_tracks:
                    seed_tracks.add(t)
                    pred_tracks[j]=t
                    j+=1
                    if j == rc:
                        break
            
        while j < rc:
            rnd_idx=np.random.permutation(tracks.shape[0])
            for idx in rnd_idx:
                if tracks.iloc[idx]['track_id'] not in seed_tracks:
                    seed_tracks.add(tracks.iloc[idx]['track_id'])
                    pred_tracks[j]=tracks.iloc[idx]['track_id']
                    j+=1
                    if j==rc:
                        break
    
        y_pred[pid_tst] = pred_tracks
#     if proc_i>-1:
#         np.save('output/title_sim_0510_%d'%proc_i,y_pred)
    
    return y_pred
    


Overwriting title.py


In [33]:
# import multiprocessing
# import title
# playlists_tra['title']=playlists_tra.name.apply(title.build_title)
# playlists_tst['title']=playlists_tst.name.apply(title.build_title)
# playlists_tra.index=playlists_tra.pid
# playlist_tracks_tra.index=playlist_tracks_tra.pid
# n=10
# slice=playlists_tst.shape[0]//n
# for i in range(n):
#     p=multiprocessing.Process(target=title.recommend, args=(playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,tracks,i,i*slice,(i+1)*slice))
#     p.start()
                              

In [34]:
y_pred,sim_cnts=recommend(playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,tracks)
r_prec(y_true,y_pred[:,1:]),ndcg(y_true,y_pred[:,1:]),clicks(y_true,y_pred[:,1:])

HBox(children=(IntProgress(value=0), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


(0.024378962155629454, 0.05946963252561297, 41.67)

## challenge

In [5]:
playlists_tra2,playlists_tst2,playlist_tracks_tra2,playlist_tracks_tst2,tracks2 = load_data()
playlists_tra2['title']=playlists_tra2.name.apply(title.build_title)
playlists_tst2['title']=playlists_tst2.name.apply(title.build_title)
playlists_tra2.index=playlists_tra2.pid
playlist_tracks_tra2.index=playlist_tracks_tra2.pid

In [15]:
recommend(pl_tra2,pl_tst2,playlist_tracks_tra2,playlist_tracks_tst2,tracks2,0,0,playlists_tst2.shape[0])

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))

spanishplaylist 55
groovin 322


/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


uplift 60
wubz 20
new 4629
calm 1347
favorite 566


KeyboardInterrupt: 

In [21]:
pl_tra2=playlists_tra2[['pid','title']]
pl_tst2=playlists_tst2[['pid','title']]

n=20
slice=playlists_tst2.shape[0]//n
for i in range(n):
    p=multiprocessing.Process(target=title.recommend, 
                              args=(pl_tra2,pl_tst2,playlist_tracks_tra2,playlist_tracks_tst2,tracks2,i,i*slice,(i+1)*slice))
    p.start()
                              

--- Logging error ---
Traceback (most recent call last):
--- Logging error ---
  File "/home/cq/anaconda3/lib/python3.6/site-packages/traitlets/traitlets.py", line 528, in get
    value = obj._trait_values[self.name]
Traceback (most recent call last):
KeyError: 'layout'

During handling of the above exception, another exception occurred:

  File "/home/cq/anaconda3/lib/python3.6/site-packages/traitlets/traitlets.py", line 528, in get
    value = obj._trait_values[self.name]
--- Logging error ---
Traceback (most recent call last):
KeyError: 'layout'
  File "/home/cq/anaconda3/lib/python3.6/logging/__init__.py", line 994, in emit
    stream.write(msg)

During handling of the above exception, another exception occurred:

OSError: [Errno 5] Input/output error
Traceback (most recent call last):
Traceback (most recent call last):
Call stack:
--- Logging error ---
  File "/home/cq/anaconda3/lib/python3.6/logging/__init__.py", line 994, in emit
    stream.write(msg)
  File "/home/cq/anaconda3/

  File "/home/cq/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
OSError: [Errno 5] Input/output error
  File "/home/cq/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 440, in _handle_events
    self._handle_recv()
  File "/home/cq/anaconda3/lib/python3.6/site-packages/traitlets/traitlets.py", line 528, in get
    value = obj._trait_values[self.name]
  File "/home/cq/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/cq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/cq/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 472, in _handle_recv
    self._run_callback(callback, msg)
Call stack:
  File "/home/cq/anaconda3/lib/py

  File "/home/cq/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 472, in _handle_recv
    self._run_callback(callback, msg)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):
  File "/home/cq/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 537, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
KeyError: 'layout'
  File "/home/cq/anaconda3/lib/python3.6/multiprocessing/context.py", line 223, in _Popen
    return _default_context.get_context().Process._Popen(process_obj)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/traitlets/traitlets.py", line 528, in get
    value = obj._trait_values[self.name]
  File "/home/cq/anaconda3/lib/python3.6/site-packages/traitlets/traitl

  File "/home/cq/anaconda3/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
Traceback (most recent call last):
--- Logging error ---
  File "/home/cq/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py", line 208, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2728, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/cq/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Call stack:

During handling of the above except

  File "/home/cq/anaconda3/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/home/cq/anaconda3/lib/python3.6/logging/__init__.py", line 994, in emit
    stream.write(msg)
  File "/home/cq/anaconda3/lib/python3.6/multiprocessing/popen_fork.py", line 80, in _launch
    code = process_obj._bootstrap()
  File "/home/cq/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/cq/projects/spotify/title.py", line 30, in recommend
    for i in tnrange(len(pids)):
  File "/home/cq/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 537, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py", line 208, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/cq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line

...

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [22]:
npys=[]
for i in range(n):
    npys.append(np.load('output/title_sim_0510_%d.npy'%i))
np.save('output/title_sim_0510',np.concatenate(npys))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [7]:
# y_pred=recommend(playlists_tra2,playlists_tst2,playlist_tracks_tra2,playlist_tracks_tst2,tracks2)
# np.save('output/title_sim_0510_1',y_pred)

In [8]:
# y_pred2=np.load('output/title_sim2.npy')
# np.save('output/title_sim',np.concatenate([y_pred,y_pred2]))